<a href="https://colab.research.google.com/github/MatheusMGirardi/py-spark-olympic-athlete-data/blob/main/PySpark_big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
import os

In [3]:
!pip install -q findspark
!pip install pyspark

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

if __name__ == "__main__":
    spark = SparkSession.builder.appName("TDE4").master("local").getOrCreate()

#### 1. CSV loading

In [5]:
df = spark.read.csv("/content/athletes.csv")
rdd = df.rdd

In [6]:
for row in rdd.take(2):
    print(row)

Row(_c0='ID', _c1='Name', _c2='Sex', _c3='Age', _c4='Height', _c5='Weight', _c6='Team', _c7='NOC', _c8='Games', _c9='Year', _c10='Season', _c11='City', _c12='Sport', _c13='Event', _c14='Medal')
Row(_c0='1', _c1='A Dijiang', _c2='M', _c3='24', _c4='180', _c5='80', _c6='China', _c7='CHN', _c8='1992 Summer', _c9='1992', _c10='Summer', _c11='Barcelona', _c12='Basketball', _c13="Basketball Men's Basketball", _c14='NA')


In [7]:
# remove o header
rdd = rdd.zipWithIndex().filter(lambda row_index: row_index[1] > 0).map(lambda row_index: row_index[0])

#### 1.1 Contagem de Atletas por Ano

In [8]:
consulta1_1m = rdd.map(lambda row: ((row._c9), 1))
consulta1_1r = consulta1_1m.reduceByKey(lambda a,b: a + b)

In [9]:
results = consulta1_1r.take(10)
for result in results:
    print(result)

('1992', 16413)
('2012', 12920)
('1920', 4292)
('1900', 1936)
('1988', 14676)
('1994', 3160)
('1932', 3321)
('2002', 4109)
('1952', 9358)
('1980', 8937)


In [10]:
output_dir = "out/consulta1_1"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

In [11]:
consulta1_1r.saveAsTextFile(output_dir)

### 1.2 Contagem de Atletas por esporte

In [12]:
consulta1_2m = rdd.map(lambda row: ((row._c12), 1))
consulta1_2r = consulta1_2m.reduceByKey(lambda a,b: a + b) #soma os val

In [13]:
results = consulta1_2r.take(10)
for result in results:
    print(result)

('Basketball', 4536)
('Judo', 3801)
('Football', 6745)
('Tug-Of-War', 170)
('Speed Skating', 5613)
('Cross Country Skiing', 9133)
('Athletics', 38624)
('Ice Hockey', 5516)
('Swimming', 23195)
('Badminton', 1457)


In [14]:
output_dir = "out/consulta1_2"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

In [15]:
consulta1_2r.saveAsTextFile(output_dir)

### 1.3 Média de Idade dos Atletas por Gênero

In [16]:
filtered_rdd = rdd.filter(lambda row: row._c3 != 'NA' and row._c3 is not None)

consulta1_3m = filtered_rdd.map(lambda row: (row._c2, (int(row._c3), 1)))

consulta1_3r = consulta1_3m.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

consulta1_3_avg = consulta1_3r.mapValues(lambda x: x[0] / x[1])

results = consulta1_3_avg.collect()

for gender, avg_age in results:
    print(f"Gender: {gender}, Average Age: {avg_age}")

Gender: M, Average Age: 26.277561532227104
Gender: F, Average Age: 23.732880779508218


In [17]:
output_dir = "out/consulta1_3"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

consulta1_3_avg.saveAsTextFile(output_dir)

### 1.4 Contagem de medalhas por país

In [18]:
consulta1_4m = rdd.filter(lambda row: row._c14 in ['Gold', 'Silver', 'Bronze']).map(lambda row: (row._c7, 1))

consulta1_4r = consulta1_4m.reduceByKey(lambda a, b: a + b)

results = consulta1_4r.take(10)

for country, total_medals in results:
    print(f"Country: {country}, Total Medals: {total_medals}")

Country: DEN, Total Medals: 597
Country: FIN, Total Medals: 900
Country: NOR, Total Medals: 1033
Country: NED, Total Medals: 1040
Country: FRA, Total Medals: 1777
Country: ITA, Total Medals: 1637
Country: ESP, Total Medals: 489
Country: AZE, Total Medals: 44
Country: RUS, Total Medals: 1165
Country: BLR, Total Medals: 139


In [19]:
output_dir = "out/consulta1_4"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

consulta1_4r.saveAsTextFile(output_dir)

### 2.1 Contagem de Medalhas por País e Ano

In [20]:
# Mapper: Emit (country, year) and 1 for each medal won (Gold, Silver, Bronze)
consulta2_1m = rdd.filter(lambda row: row._c14 in ['Gold', 'Silver', 'Bronze']).map(lambda row: ((row._c7, row._c9), 1))

consulta2_1r = consulta2_1m.reduceByKey(lambda a, b: a + b)

results = consulta2_1r.take(10)

# Display the results
for (country, year), total_medals in results:
    print(f"Country: {country}, Year: {year}, Total Medals: {total_medals}")

Country: DEN, Year: 1900, Total Medals: 9
Country: FIN, Year: 1920, Total Medals: 52
Country: FIN, Year: 2014, Total Medals: 33
Country: FIN, Year: 1948, Total Medals: 42
Country: FIN, Year: 1952, Total Medals: 52
Country: NOR, Year: 1992, Total Medals: 47
Country: NOR, Year: 1994, Total Medals: 30
Country: NOR, Year: 2002, Total Medals: 41
Country: NOR, Year: 2006, Total Medals: 23
Country: NOR, Year: 2008, Total Medals: 22


In [21]:
output_dir = "out/consulta2_1"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

consulta2_1r.saveAsTextFile(output_dir)

### 2.2. Top 3 Atletas com Mais Medalhas

In [22]:
# Mapper: Emit (atleta, 1) for each medal won (Gold, Silver, Bronze)
consulta2_2m = rdd.filter(lambda row: row._c14 in ['Gold', 'Silver', 'Bronze']).map(lambda row: (row._c1, 1))

consulta2_2r = consulta2_2m.reduceByKey(lambda a, b: a + b)

# Get the top 3 athletes with the most medals
top_3_atletas = consulta2_2r.takeOrdered(3, key=lambda x: -x[1])

for atleta, total_medals in top_3_atletas:
    print(f"Atleta: {atleta}, Total Medals: {total_medals}")

Atleta: Michael Fred Phelps II, Total Medals: 28
Atleta: Larysa Semenivna Latynina (Diriy-), Total Medals: 18
Atleta: Nikolay Yefimovich Andrianov, Total Medals: 15


In [23]:
output_dir = "out/consulta2_2"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)


top_3_atletas_rdd = spark.sparkContext.parallelize(top_3_atletas)
top_3_atletas_rdd.saveAsTextFile(output_dir)

### 2.3. Contagem de Atletas por Faixa Etária e Esporte

In [24]:
def categorize_age(age):
    if 20 <= age <= 30:
        return '20-30'
    elif 31 <= age <= 40:
        return '31-40'
    else:
        return 'Other'

filtered_rdd = rdd.filter(lambda row: row._c3.isdigit())

consulta2_3m = filtered_rdd.map(lambda row: ((categorize_age(int(row._c3)), row._c12), 1))

consulta2_3r = consulta2_3m.reduceByKey(lambda a, b: a + b)

results = consulta2_3r.take(10)

for (faixa_etaria, esporte), total_athletes in results:
    print(f"Faixa Etária: {faixa_etaria}, Esporte: {esporte}, Total Atletas: {total_athletes}")

Faixa Etária: 20-30, Esporte: Basketball, Total Atletas: 3712
Faixa Etária: 20-30, Esporte: Judo, Total Atletas: 3187
Faixa Etária: 20-30, Esporte: Football, Total Atletas: 5549
Faixa Etária: 31-40, Esporte: Tug-Of-War, Total Atletas: 42
Faixa Etária: 20-30, Esporte: Speed Skating, Total Atletas: 4534
Faixa Etária: 31-40, Esporte: Cross Country Skiing, Total Atletas: 1339
Faixa Etária: Other, Esporte: Athletics, Total Atletas: 2754
Faixa Etária: 20-30, Esporte: Ice Hockey, Total Atletas: 4377
Faixa Etária: 20-30, Esporte: Swimming, Total Atletas: 12613
Faixa Etária: 20-30, Esporte: Cross Country Skiing, Total Atletas: 7318


In [25]:
output_dir = "out/consulta2_3"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

consulta2_3r.saveAsTextFile(output_dir)

### 2.4. Proporção de Atletas Homens e Mulheres por Ano

In [26]:
consulta2_4m = rdd.map(lambda row: (row._c9, (1, 0)) if row._c2 == 'M' else (row._c9, (0, 1)))

# Combine sum of (males, females) by year
consulta2_4c = consulta2_4m.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

# Reducer:  sum of (males, females) by year and calculate the proportion
consulta2_4r = consulta2_4c.mapValues(lambda counts: (counts[0], counts[1], counts[0] / (counts[0] + counts[1]), counts[1] / (counts[0] + counts[1])))

results = consulta2_4r.take(10)

for year, (males, females, male_ratio, female_ratio) in results:
    print(f"Ano: {year}, Homens: {males}, Mulheres: {females}, Proporção Homens: {male_ratio:.2f}, Proporção Mulheres: {female_ratio:.2f}")

Ano: 1992, Homens: 11235, Mulheres: 5178, Proporção Homens: 0.68, Proporção Mulheres: 0.32
Ano: 2012, Homens: 7105, Mulheres: 5815, Proporção Homens: 0.55, Proporção Mulheres: 0.45
Ano: 1920, Homens: 4158, Mulheres: 134, Proporção Homens: 0.97, Proporção Mulheres: 0.03
Ano: 1900, Homens: 1903, Mulheres: 33, Proporção Homens: 0.98, Proporção Mulheres: 0.02
Ano: 1988, Homens: 10453, Mulheres: 4223, Proporção Homens: 0.71, Proporção Mulheres: 0.29
Ano: 1994, Homens: 2055, Mulheres: 1105, Proporção Homens: 0.65, Proporção Mulheres: 0.35
Ano: 1932, Homens: 2952, Mulheres: 369, Proporção Homens: 0.89, Proporção Mulheres: 0.11
Ano: 2002, Homens: 2527, Mulheres: 1582, Proporção Homens: 0.61, Proporção Mulheres: 0.39
Ano: 1952, Homens: 7676, Mulheres: 1682, Proporção Homens: 0.82, Proporção Mulheres: 0.18
Ano: 1980, Homens: 6751, Mulheres: 2186, Proporção Homens: 0.76, Proporção Mulheres: 0.24


In [27]:
output_dir = "out/consulta2_4"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

consulta2_4r.saveAsTextFile(output_dir)

### 3.1. Evolução da Idade Média dos Atletas por País

In [28]:
filtered_rdd = rdd.filter(lambda row: row._c3.isdigit())

# Mapper: Emit ((country, year), (age, 1)) for each row
consulta3_1m = filtered_rdd.map(lambda row: ((row._c6, row._c9), (float(row._c3), 1)))

# Reducer: Sum ages and counts
consulta3_1c = consulta3_1m.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

# average age
consulta3_1r = consulta3_1c.mapValues(lambda x: x[0] / x[1])

# Mapper: Emit (country, (year, average_age)) for each entry
consulta3_1m2 = consulta3_1r.map(lambda x: (x[0][0], (x[0][1], x[1])))

results = consulta3_1m2.take(10)

for country, (year, avg_age) in results:
    print(f"Country: {country}, Year: {year}, Average Age: {avg_age:.2f}")

Country: China, Year: 1992, Average Age: 21.96
Country: China, Year: 2012, Average Age: 23.89
Country: Denmark, Year: 1920, Average Age: 28.83
Country: Denmark/Sweden, Year: 1900, Average Age: 30.83
Country: Netherlands, Year: 1988, Average Age: 24.83
Country: Netherlands, Year: 1992, Average Age: 25.01
Country: Netherlands, Year: 1994, Average Age: 22.67
Country: United States, Year: 1992, Average Age: 25.47
Country: United States, Year: 1994, Average Age: 24.98
Country: Netherlands, Year: 1932, Average Age: 37.56


In [29]:
output_dir = "out/consulta3_1"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

consulta3_1m2.saveAsTextFile(output_dir)

### 3.2 Contagem de Medalhas por Tipo (Ouro, Prata, Bronze) por País e Ano

In [30]:
medal_counts_map = rdd.filter(lambda row: row._c14.lower() in ["gold", "silver", "bronze"]).map(lambda row: ((row._c7, row._c9, row._c14.lower()), 1))

medal_counts_reduce = medal_counts_map.reduceByKey(lambda a, b: a + b)

medal_counts_by_type = medal_counts_reduce.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1]))

total_medals_map = medal_counts_by_type.map(lambda x: ((x[0], x[1]), (x[2], x[3])))

total_medals_reduce = total_medals_map.groupByKey().mapValues(lambda medals: {
    "total": sum(count for _, count in medals),
    "gold_percent": sum(count for medal, count in medals if medal == "gold") / sum(count for _, count in medals),
    "silver_percent": sum(count for medal, count in medals if medal == "silver") / sum(count for _, count in medals),
    "bronze_percent": sum(count for medal, count in medals if medal == "bronze") / sum(count for _, count in medals)
})

# Formatted result
results = total_medals_reduce.take(10)
for (country, year), data in results:
    print(f"País: {country}, Ano: {year}, Total: {data['total']}, Percentual Ouro: {data['gold_percent']:.2%}, Percentual Prata: {data['silver_percent']:.2%}, Percentual Bronze: {data['bronze_percent']:.2%}")


País: DEN, Ano: 1900, Total: 9, Percentual Ouro: 44.44%, Percentual Prata: 33.33%, Percentual Bronze: 22.22%
País: FIN, Ano: 1920, Total: 52, Percentual Ouro: 40.38%, Percentual Prata: 26.92%, Percentual Bronze: 32.69%
País: FIN, Ano: 2014, Total: 33, Percentual Ouro: 6.06%, Percentual Prata: 21.21%, Percentual Bronze: 72.73%
País: FIN, Ano: 1948, Total: 42, Percentual Ouro: 42.86%, Percentual Prata: 33.33%, Percentual Bronze: 23.81%
País: FIN, Ano: 1952, Total: 52, Percentual Ouro: 26.92%, Percentual Prata: 13.46%, Percentual Bronze: 59.62%
País: NOR, Ano: 1992, Total: 47, Percentual Ouro: 23.40%, Percentual Prata: 63.83%, Percentual Bronze: 12.77%
País: NOR, Ano: 1994, Total: 30, Percentual Ouro: 26.67%, Percentual Prata: 56.67%, Percentual Bronze: 16.67%
País: NOR, Ano: 2002, Total: 41, Percentual Ouro: 56.10%, Percentual Prata: 26.83%, Percentual Bronze: 17.07%
País: NOR, Ano: 2006, Total: 23, Percentual Ouro: 8.70%, Percentual Prata: 39.13%, Percentual Bronze: 52.17%
País: NOR, An

In [31]:
output_dir = "out/consulta3_2"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

consulta3_1m2.saveAsTextFile(output_dir)